In [1]:
import pandas as pd
import requests
import time
import random
from tqdm import tqdm
from bs4 import BeautifulSoup

cookies = {
    'TIKI_GUEST_TOKEN': '8jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY',
    'TOKENS': '{%22access_token%22:%228jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY%22%2C%22expires_in%22:157680000%2C%22expires_at%22:1763654224277%2C%22guest_token%22:%228jWSuIDBb2NGVzr6hsUZXpkP1FRin7lY%22}',
    'amp_99d374': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlohtdv.3.2.5',
    'amp_99d374_tiki.vn': 'eSc-_0HT1um7cb57E7dwA0...1enloc6a2.1enlocds8.0.1.1',
    '_gcl_au': '1.1.559117409.1605974236',
    '_ants_utm_v2': '',
    '_pk_id.638735871.2fc5': 'b92ae025fbbdb31f.1605974236.1.1605974420.1605974236.',
    '_pk_ses.638735871.2fc5': '*',
    '_trackity': '70e316b0-96f2-dbe1-a2ed-43ff60419991',
    '_ga_NKX31X43RV': 'GS1.1.1605974235.1.1.1605974434.0',
    '_ga': 'GA1.1.657946765.1605974236',
    'ai_client_id': '11935756853.1605974227',
    'an_session': 'zizkzrzjzlzizqzlzqzjzdzizizqzgzmzkzmzlzrzmzgzdzizlzjzmzqzkznzhzhzkzdzhzdzizlzjzmzqzkznzhzhzkzdzizlzjzmzqzkznzhzhzkzdzjzdzhzqzdzizd2f27zdzjzdzlzmzmznzq',
    'au_aid': '11935756853',
    'dgs': '1605974411%3A3%3A0',
    'au_gt': '1605974227146',
    '_ants_services': '%5B%22cuid%22%5D',
    '__admUTMtime': '1605974236',
    '__iid': '749',
    '__su': '0',
    '_bs': 'bb9a32f6-ab13-ce80-92d6-57fd3fd6e4c8',
    '_gid': 'GA1.2.867846791.1605974237',
    '_fbp': 'fb.1.1605974237134.1297408816',
    '_hjid': 'f152cf33-7323-4410-b9ae-79f6622ebc48',
    '_hjFirstSeen': '1',
    '_hjIncludedInPageviewSample': '1',
    '_hjAbsoluteSessionInProgress': '0',
    '_hjIncludedInSessionSample': '1',
    'tiki_client_id': '657946765.1605974236',
    '__gads': 'ID=ae56424189ecccbe-227eb8e1d6c400a8:T=1605974229:RT=1605974229:S=ALNI_MZFWYf2BAjzCSiRNLC3bKI-W_7YHA',
    'proxy_s_sv': '1605976041662',
    'TKSESSID': '8bcd49b02e1e16aa1cdb795c54d7b460',
    'TIKI_RECOMMENDATION': '21dd50e7f7c194df673ea3b717459249',
    '_gat': '1',
    'cto_bundle': 'i6f48l9NVXNkQmJ6aEVLcXNqbHdjcVZoQ0k2clladUF2N2xjZzJ1cjR6WG43UTVaRmglMkZXWUdtRnJTNHZRbmQ4SDAlMkZwRFhqQnppRHFxJTJCSEozZXBqRFM4ZHVxUjQ2TmklMkJIcnhJd3luZXpJSnBpcE1nJTNE',
    'TIKI_RECENTLYVIEWED': '58259141',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'vi,vi-VN;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5',
    'Referer': 'https://tiki.vn/may-chieu-mini-beecube-x2-max-gen-3-2023-hang-chinh-hang-p274032002.html?itm_campaign=CTP_YPD_TKA_PLA_UNK_ALL_UNK_UNK_UNK_UNK_X.195124_Y.1777451_Z.3915926_CN.X2-max-Gen-4&itm_medium=CPC&itm_source=tiki-ads&spid=274032006',
    'X-Guest-Token': '5kBWiz8uXp20s4bSoIE3aNRUTwvMOLtd',
    'Connection': 'keep-alive',
    'TE': 'Trailers',
}

params = (
    ('platform', 'web'),
    ('spid', 187266106)
    #('include', 'tag,images,gallery,promotions,badges,stock_item,variants,product_links,discount_tag,ranks,breadcrumbs,top_features,cta_desktop'),
)

def parser_product(json, html):
    d = dict()
    d['id'] = json.get('id')
    d['sku'] = json.get('sku')
    d['short_description'] = json.get('short_description')
    d['price'] = json.get('price')
    d['list_price'] = json.get('list_price')
    d['price_usd'] = json.get('price_usd')
    d['discount'] = json.get('discount')
    d['discount_rate'] = json.get('discount_rate')
    d['review_count'] = json.get('review_count')
    d['order_count'] = json.get('order_count')
    d['inventory_status'] = json.get('inventory_status')
    d['is_visible'] = json.get('is_visible')
    d['stock_item_qty'] = json.get('stock_item').get('qty')
    d['stock_item_max_sale_qty'] = json.get('stock_item').get('max_sale_qty')
    d['product_name'] = json.get('meta_title')
    d['brand_id'] = json.get('brand').get('id')
    d['brand_name'] = json.get('brand').get('name')

    # Parse the HTML to extract image URLs
    soup = BeautifulSoup(html, 'html.parser')
    images = []
    for picture in soup.find_all('picture', class_='webpimg-container'):
        for source in picture.find_all('source'):
            img_url = source.get('srcset')
            if img_url:
                images.append(img_url)
        img = picture.find('img')
        if img and img.get('srcset'):
            images.append(img.get('srcset'))

    # Deduplicate and clean up the image URLs
    images = list(set(images))
    d['images'] = images

    return d



In [3]:
# # Read product IDs from CSV
# df_id = pd.read_csv('product_id_ncds1.csv')
# p_ids = df_id.id.to_list()
# result = []
# for pid in tqdm(p_ids, total=len(p_ids)):
#     response = requests.get(f'https://tiki.vn/api/v2/products/{pid}', headers=headers, params=params, cookies=cookies)
#     if response.status_code == 200:
#         print(f'Crawl data {pid} success !!!')
#         json_data = response.json()

#         # Fetch HTML content
#         html_response = requests.get(f'https://tiki.vn/product/{pid}', headers=headers, params=params, cookies=cookies)
#         if html_response.status_code == 200:
#             html_content = html_response.text
#             result.append(parser_product(json_data, html_content))
#         else:
#             print(f'Failed to fetch HTML for product {pid}')

# Save results to CSV
df_product = pd.DataFrame(result)
df_product.to_csv('crawled_data_ncds1.csv', index=False)

In [ ]:
!pip install pandas openpyxl
import pandas as pd

# Đọc dữ liệu từ tệp CSV
csv_file_path = 'crawled_data_ncds.csv'
data = pd.read_csv(csv_file_path)

# Ghi dữ liệu vào tệp Excel
excel_file_path = 'crawled_data_ncds1.xlsx'
data.to_excel(excel_file_path, index=False)

print(f'Dữ liệu đã được chuyển đổi thành công từ {csv_file_path} sang {excel_file_path}')


Dữ liệu đã được chuyển đổi thành công từ crawled_data_ncds.csv sang crawled_data_ncds.xlsx
